In [10]:
%pylab nbagg
from tvb.simulator.lab import *
import os.path
from matplotlib import colors, cm
import time
import scipy.signal as sig

import numpy as np
import pandas as pd
import scipy.io
import scipy

# downloaded library for peak detection in z time series
import peakdetect
from tvbsim.util import *

from runmainsim import *

%load_ext autoreload
%autoreload 2

Populating the interactive namespace from numpy and matplotlib
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Analyze Region and Test Functions To Modify Contacts

In [13]:
patient = 'id001_ac'
project_dir = '/Users/adam2392/Documents/tvb/metadata/'
confile = os.path.join(project_dir, patient, "connectivity.zip")
####################### 1. Extract Relevant Info ########################
con = extractcon(confile)
region_centers = con.centres
regions = con.region_labels
seegfile = os.path.join(project_dir, patient, "seeg.txt")
seeg_xyz = extractseegxyz(seegfile)

# first get all contacts of the same electrode
seeg_labels = np.array(seeg_xyz.index, dtype='str')

# determine closest contact for region
ezregion = ['ctx-lh-bankssts']
ezindice, pzindice = getindexofregion(regions, ezregion)
near_seeg = findclosestcontact(ezindice, region_centers, seeg_xyz)

# now move contact and recompute gain matrix
seeg_contact = np.array(seeg_xyz.iloc[near_seeg[1]].index, dtype='str')[0]
electrodeindices = getallcontacts(seeg_labels, seeg_contact)

new_seeg_xyz = movecontact(seeg_xyz, region_centers, ezindice, seeg_contact)

gainmat = simplest_gain_matrix(new_seeg_xyz.as_matrix(), reg_xyz=region_centers)

print gainmat.shape
print seeg_contact
print seeg_xyz.iloc[electrodeindices]
print new_seeg_xyz.iloc[electrodeindices]

# print near_seeg[1].ravel()
print seeg_xyz.iloc[near_seeg[1]]

is left
is left
(70, 84)
OR'1
           x      y      z
OR'1  -17.97  -4.23   4.62
OR'2  -18.36  -2.37   7.56
OR'3  -18.75  -0.52  10.50
OR'4  -19.13   1.34  13.45
OR'5  -19.52   3.20  16.39
OR'6  -19.90   5.05  19.33
OR'7  -20.29   6.91  22.27
OR'8  -20.68   8.76  25.22
OR'9  -21.06  10.62  28.16
OR'10 -21.45  12.47  31.10
OR'11 -21.84  14.33  34.04
OR'12 -22.22  16.18  36.99
OR'13 -22.61  18.04  39.93
OR'14 -22.99  19.90  42.87
             x        y        z
OR'1   10.4662  59.5403  -5.6749
OR'2   10.0762  61.4003  -2.7349
OR'3    9.6862  63.2503   0.2051
OR'4    9.3062  65.1103   3.1551
OR'5    8.9162  66.9703   6.0951
OR'6    8.5362  68.8203   9.0351
OR'7    8.1462  70.6803  11.9751
OR'8    7.7562  72.5303  14.9251
OR'9    7.3762  74.3903  17.8651
OR'10   6.9862  76.2403  20.8051
OR'11   6.5962  78.1003  23.7451
OR'12   6.2162  79.9503  26.6951
OR'13   5.8262  81.8103  29.6351
OR'14   5.4462  83.6703  32.5751
          x     y     z
OR'1 -17.97 -4.23  4.62


# Summary

Here, I created a suite of utility functions for modifying the simulation paradigm by moving contacts around based on the ezregion specified. In addition, it will move the entire contact and create a new very simplified gain matrix.

This can all be plugged back into the corresponding tvb simulation.

To Do:
* Moving contacts to within a specified distance from ez region
* Get the close by regions within a distance for a seeg contact
* Constrain the movement to make sure implantation is realistic? It is possible for contacts to be very close to other electrode's contacts right now upon artificial movement.